In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls

gdrive	sample_data


In [0]:
# We can print the first rows to get an idea of what the format looks like using the head linux command
!head -n 1 'gdrive/My Drive/Deep Learning for Mental Health/Bellevue/socioeconomic_data_v2.csv'

"FIPS","Geographic Identifier","Name of Area","Qualifying Name","State/U.S.-Abbreviation (USPS)","Summary Level","Geographic Component","File Identification","Logical Record Number","US","Region","Division","State (Census Code)","State (FIPS)","County","County Subdivision (FIPS)","Place (FIPS Code)","Place (State FIPS + Place FIPS)","Census Tract","Block Group","Consolidated City","American Indian Area/Alaska Native Area/Hawaiian Home Land (Census)","American Indian Area/Alaska Native Area/Hawaiian Home Land (FIPS)","American Indian Trust Land/Hawaiian Home Land Indicator","American Indian Tribal Subdivision (Census)","American Indian Tribal Subdivision (FIPS)","Alaska Native Regional Corporation (FIPS)","Metropolitan and Micropolitan Statistical Area","Combined Statistical Area","Metropolitan Division","Metropolitan Area Central City","Metropolitan/Micropolitan Indicator Flag","New England City and Town Combined Statistical Area","New England City and Town Area","New England City and 

In [0]:
import pandas as pd
import csv
import numpy as np

In [0]:
root_dir = "gdrive/My Drive/Deep Learning for Mental Health/"

In [0]:
# Read mental health file from 500 cities project
tractids = []
obvalues = {}
def populate(city, filename):
    tractids.clear()
    tractind = 0
    levelind = 0
    dataind = 0
    with open(root_dir + city + "/" + filename, 'r') as f:
        obreader = csv.reader(f)
        header = next(obreader)

        for i in range(0, len(header)):
            if header[i] == 'TractFIPS':
                tractind = i
                continue
            if header[i] == 'GeographicLevel':
                levelind = i
                continue
            if header[i] == 'Data_Value':
                dataind = i
                continue
        for row in obreader:
            if row[levelind] != 'Census Tract':
                continue
            if row[dataind] == '':
                # print(row)
                continue
            tractids.append(row[tractind])
            obvalues[row[tractind]] = row[dataind]
    print('Total number of census tracts in datafile: ', len(tractids))

In [0]:
def normalize(value, total):
    try:
        result = float(value)/int(total)
    except (ZeroDivisionError, KeyError) as e:
        result = 0
    return result

In [0]:
"""
0 FIPS
56 Total Population
57 Population Density (Per Sq. Mile)
60 Total Population: Male
61 Total Population: Female
67 Total Population: 18 to 24 Years
68 Total Population: 25 to 34 Years
69 Total Population: 35 to 44 Years
70 Total Population: 45 to 54 Years
71 Total Population: 55 to 64 Years
72 Total Population: 65 to 74 Years
73 Total Population: 75 to 84 Years
74 Total Population: 85 Years and Over
75 Households:
76 Households: Family Households
77 Households: Family Households: Married-Couple Family
78 Households: Family Households: Other Family
79 Households: Family Households: Other Family: Male Householder, No Wife Present
80 Households: Family Households: Other Family: Female Householder, No Husband Present
81 Households: Nonfamily Households
82 Households: Nonfamily Households: Male Householder
83 Households: Nonfamily Households: Female Householder
84 Population 16 Years and Over:
85 Population 16 Years and Over: in Labor Force
86 Population 16 Years and Over: in Labor Force: in Armed Forces
87 Population 16 Years and Over: in Labor Force: Civilian
88 Population 16 Years and Over: in Labor Force: Civilian: Employed
89 Population 16 Years and Over: in Labor Force: Civilian: Unemployed
90 Population 16 Years and Over: Not in Labor Force
91 Employed Civilian Population 16 Years and Over:
92 Employed Civilian Population 16 Years and Over: Management, Business, and Financial Operations  Occupations
93 Employed Civilian Population 16 Years and Over: Professional and Related Occupations
94 Employed Civilian Population 16 Years and Over: Healthcare Support Occupations
95 Employed Civilian Population 16 Years and Over: Protective Service Occupations
96 Employed Civilian Population 16 Years and Over: Food Preparation and Serving Related Occupations
97 Employed Civilian Population 16 Years and Over: Building and Grounds Cleaning and Maintenance  Occupations
98 Employed Civilian Population 16 Years and Over: Personal Care and Service Occupations
99 Employed Civilian Population 16 Years and Over: Sales and Related Occupations
100 Employed Civilian Population 16 Years and Over: Office and Administrative Support Occupations
101 Employed Civilian Population 16 Years and Over: Farming, Fishing, and Forestry Occupations
102 Employed Civilian Population 16 Years and Over: Construction, Extraction, and Maintenance  Occupations
103 Employed Civilian Population 16 Years and Over: Production Occupations
104 Employed Civilian Population 16 Years and Over: Transportation and Material Moving Occupations
105 Per Capita Income (In 2017 Inflation Adjusted Dollars)
106 Gini Index
108 Total Population: Native Born
109 Total Population: Foreign Born
110 Total Population: Foreign Born: Naturalized Citizen
111 Total Population: Foreign Born: Not a Citizen
"""
def extract(city, filename):
    with open(root_dir + city + "/" + filename, 'r') as file:
        file_reader = csv.reader(file)

        # human readable column names
        _ = next(file_reader)

        _ = next(file_reader) # column codes    

        tractids_model = []
        features = []
        for row in file_reader:
            # ignore rows
            if row[0] not in tractids:
                continue
            
            feature = []
            
            # population density
            try:
              feature.append(float(row[57])) 
            except ValueError as e:
              feature.append(0)

            # male and female 
            feature.append(normalize(row[60], row[56])) 
            feature.append(normalize(row[61], row[56]))

            # different age-groups above 18
            feature.append(normalize(row[67], row[56])) # 18-24
            feature.append(normalize(row[68], row[56])) # 25-34
            feature.append(normalize(row[69], row[56])) # 35-44
            feature.append(normalize(row[70], row[56])) # 45-54
            feature.append(normalize(row[71], row[56])) # 55-64
            feature.append(normalize(row[72], row[56])) # 65-74
            feature.append(normalize(row[73], row[56])) # 75-84
            feature.append(normalize(row[74], row[56])) # 85+
            
            # family household and non-family household
            feature.append(normalize(row[76], row[75]))
            feature.append(normalize(row[81], row[75]))

            # labor force and non-labor force
            feature.append(normalize(row[85], row[84]))
            feature.append(normalize(row[90], row[84]))

            # armed labor forces
            feature.append(normalize(row[86], row[85]))

            # unemployed labor force
            feature.append(normalize(row[89], row[87]))

            # per capita income 
            try:
              feature.append(float(row[105]))
            except ValueError as e:
              feature.append(0)
            
            # gini index
            try:
              feature.append(float(row[106]))
            except ValueError as e:
              feature.append(0)

            # native and foreign born
            feature.append(normalize(row[108], row[56]))
            feature.append(normalize(row[109], row[56]))
            
            features.append(feature)
            tractids_model.append(row[0])

        print("Total number of census tracts with extracted features: ", len(features))

        np.save((root_dir + city + "/extracted_socio_" + city + "_features.npy"), features)
        np.save((root_dir + city + "/extracted_socio_" + city + "_tracts.npy"), tractids_model)

Extract socioeconomic features for Bellevue 

In [55]:
# load data into a pandas DataFrame
df_raw_data = pd.read_csv(root_dir + "Bellevue/socioeconomic_data_v2.csv")
df_raw_data.shape

(1459, 112)

In [77]:
populate("Bellevue", "500_cities_Bellevue_mental_health.csv")

Total number of census tracts in datafile:  28


In [78]:
extract("Bellevue", "socioeconomic_data_v2.csv")

Total number of census tracts with extracted features:  28


Extract socioeconomic features for LA

In [58]:
# load data into a pandas DataFrame 
df_raw_data = pd.read_csv(root_dir + "LA/socioeconomic_data_la.csv")
df_raw_data.shape

(2347, 112)

In [79]:
populate("LA", "500_cities_lacity_mental_health.csv")

Total number of census tracts in datafile:  994


In [80]:
extract("LA", "socioeconomic_data_la.csv")

Total number of census tracts with extracted features:  993


Extract socioeconomic features for San Antonio

In [81]:
# load data into a pandas DataFrame 
df_raw_data = pd.read_csv(root_dir + "SA/socioeconomic_census_texas.csv")
df_raw_data.shape

(5266, 112)

In [83]:
populate("SA", "500_cities_San_Antonio_mental_health.csv")

Total number of census tracts in datafile:  311


In [84]:
extract("SA", "socioeconomic_census_texas.csv")

Total number of census tracts with extracted features:  311


Extract socioeconomic features for Memphis


In [87]:
# load data into a pandas DataFrame 
df_raw_data = pd.read_csv(root_dir + "Memphis/socioeconomic_census_tenesse.csv")
df_raw_data.shape

(1498, 112)

In [90]:
populate("Memphis", "500_cities_Memphis_mental_health.csv")

Total number of census tracts in datafile:  178


In [91]:
extract("Memphis", "socioeconomic_census_tenesse.csv")

Total number of census tracts with extracted features:  178
